# Laboratorium 4 - rekomendacje dla portali informacyjnych

## Przygotowanie

 * pobierz i wypakuj dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * więcej możesz poczytać tutaj: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab4`
 * zainstaluj potrzebne biblioteki:
 `pip install nltk sklearn`

## Część 1 - preprocessing

In [1]:
# importujemy wszystkie potrzebne pakiety

import codecs
from collections import defaultdict # mozesz uzyc zamiast zwyklego slownika, rozwaz wplyw na czas obliczen
import math
import re
from string import punctuation

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# mozesz uzyc do obliczania najbardziej podobnych tekstow zamiast liczenia "na piechote"
# ale pamietaj o dostosowaniu formatu danych
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
PATH = '.'
STOPWORDS = set(stopwords.words('english'))

In [3]:
def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split('\t')[:5]
    return {
        'news_id': news_id,
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'abstract': abstract
    }

def get_news_metadata():
    with codecs.open(f'{PATH}/news.tsv', 'r', 'UTF-8') as f:
        raw = [x for x in f.read().split('\n') if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x['news_id']: x for x in parsed_entries}

news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


#### Stemming - Sprowadzamy słowa w całym korpusie do form podstawowych, usunięcie stop-words itp.

In [5]:
from string import punctuation

def preprocess_text(text):
    text = ' '.join(text.split())
    text = text.translate(str.maketrans('', '', punctuation))
    text = ''.join([i for i in text if not i.isdigit()])
    text = text.lower()
    text = text.split()
    text = [w for w in text if not w in STOPWORDS]
    
    return text
    
def stem_texts(corpus):
    stemmer = SnowballStemmer(language='english') # przetestuj rozne stemmery
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]

texts = [news[news_id]['abstract'] for news_id in news_ids]
stemmed_texts = stem_texts(texts)

In [8]:
print(texts[2] + '\n')
print(' '.join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

think realli good team team realli special good thing group close brian schmetzer


In [9]:
def get_all_words_sorted(corpus):
    result = []
    for text in corpus:
        result.extend(text)
    result = sorted(set(result))
    return result

wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

41849


## Część 2 - Bag of words
#### Obliczamy wartości Bag of Words, czyli ile razy dane slowo wystepuje w danym tekscie oraz w ilu tekstach wystapilo dane slowo

In [11]:
def get_document_frequencies(corpus, wordlist):
    document_frequencies = {word: 0 for word in wordlist}
    for text in corpus:
        for word in set(text):
            document_frequencies[word] += 1
    return document_frequencies

document_frequency = get_document_frequencies(stemmed_texts, wordlist)
document_frequency

{'aa': 8,
 'aaa': 21,
 'aaaa': 1,
 'aaaaaaaand': 1,
 'aaaaaand': 1,
 'aaaaah': 1,
 'aac': 8,
 'aaf': 2,
 'aagdzga': 1,
 'aah': 2,
 'aalia': 2,
 'aaliyah': 2,
 'aamir': 1,
 'aanchal': 1,
 'aannounc': 1,
 'aap': 2,
 'aapl': 1,
 'aarni': 1,
 'aarnseth': 1,
 'aaron': 129,
 'aarp': 3,
 'aau': 3,
 'ab': 19,
 'aba': 2,
 'ababi': 1,
 'ababneh': 1,
 'aback': 3,
 'abadi': 1,
 'abajustin': 1,
 'abalon': 2,
 'abandon': 95,
 'abar': 1,
 'abat': 2,
 'abba': 1,
 'abbasov': 1,
 'abbevill': 2,
 'abbey': 4,
 'abbi': 7,
 'abbott': 18,
 'abc': 90,
 'abcth': 1,
 'abcv': 1,
 'abdelaziz': 1,
 'abdelkad': 4,
 'abdi': 3,
 'abdic': 1,
 'abdihakim': 2,
 'abdomen': 7,
 'abdomin': 8,
 'abduct': 25,
 'abdul': 2,
 'abduljabbar': 1,
 'abdullah': 3,
 'abdulmateen': 1,
 'abdulwahe': 1,
 'abdurrahman': 1,
 'abe': 7,
 'abel': 2,
 'abellard': 5,
 'abelseth': 2,
 'abercrombi': 1,
 'aberdeen': 4,
 'abhijit': 2,
 'abhorr': 1,
 'abid': 3,
 'abigail': 10,
 'abil': 90,
 'abimelec': 1,
 'abington': 2,
 'abita': 1,
 'abject': 1,


In [12]:
def get_term_frequencies(corpus, news_indices):
    term_frequencies = {news_id: {} for news_id in news_indices.keys()}
    for news_id, text in zip(news_indices, corpus):
        for word in text:
            term_frequencies[news_id][word] = term_frequencies[news_id].get(word, 0) + 1
    return term_frequencies

term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [13]:
term_frequency[news_ids[2]]

{'think': 1,
 'realli': 2,
 'good': 2,
 'team': 2,
 'special': 1,
 'thing': 1,
 'group': 1,
 'close': 1,
 'brian': 1,
 'schmetzer': 1}

## Część 3 - TF-IDF
#### Obliczamy macierz wartości TF-IDF na podstawie bag of words

In [14]:
def calculate_tf_idf(term_frequency, document_frequency, corpus_size):
    tf_idf = {news_id: {word: 0 for word in term_frequency[news_id].keys()} for news_id in term_frequency.keys()}
    for news_id in term_frequency.keys():
        for word in term_frequency[news_id].keys():
            tf_idf[news_id][word] = term_frequency[news_id][word] * math.log(corpus_size/document_frequency[word])
    return tf_idf

tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))
tf_idf

{'N10': {'fartlek': 10.845095092394073},
 'N100': {'washington': 4.148060844727588,
  'somewher': 7.348587530927593,
  'eastern': 5.727101279977318,
  'sky': 5.861488470685736,
  'astro': 4.843680214432923,
  'assembl': 6.582415215352758,
  'one': 2.526840763595228,
  'airplan': 7.107425474110705,
  'nation': 3.449373483792028,
  'capit': 5.566980433163556,
  'team': 3.078678194374417,
  'confer': 4.8944525398063465,
  'conundrum': 10.151947911834128,
  'playerson': 10.151947911834128,
  'meet': 4.480344335542162,
  'wednesday': 3.176533984378176,
  'wretch': 10.845095092394073,
  'loss': 4.164240413603858,
  'game': 2.884771463245233,
  'run': 3.845672624886112,
  'two': 2.8017529219524646,
  'veteran': 4.314217464668188,
  'calm': 7.016453695904978,
  'concern': 5.052081484009929,
  'crop': 7.083894976700511,
  'justin': 5.668945359820244,
  'verland': 6.837761907161602,
  'jose': 6.426254484597475,
  'altuv': 7.2897470309046595,
  'emphas': 7.900656113227632,
  'encourag': 6.4506459

In [15]:
tf_idf[news_ids[2]]

{'think': 4.360459856758821,
 'realli': 9.707261090572182,
 'good': 8.085400658139525,
 'team': 6.157356388748834,
 'special': 4.765161897298483,
 'thing': 4.111693200556713,
 'group': 4.261685870235309,
 'close': 3.9244235881453897,
 'brian': 5.624739267315748,
 'schmetzer': 9.458800731274183}

## Część 4 - Podobieństwo tekstów
#### Obliczamy macierz prawdopodobieństwa (np. metryka cosinusowa)

In [20]:
def calculate_distance(tf_idf, id1, id2):
    distance = 0
    
    # metryka euklidesowa
    for word in set(tf_idf[id1].keys()) | set(tf_idf[id2].keys()):
        val1 = tf_idf[id1].get(word, 0)
        val2 = tf_idf[id2].get(word, 0)
        distance += (val1 - val2)**2
    distance = math.sqrt(distance)
    
#     # metryka cosinusowa
#     val1_norm = 1
#     val2_norm = 1

#     for word in set(tf_idf[id1].keys()) | set(tf_idf[id2].keys()):
#         val1 = tf_idf[id1].get(word, 0)
#         val2 = tf_idf[id2].get(word, 0)

#         distance += val1 * val2

#         val1_norm += val1 ** 2
#         val2_norm += val2 ** 2

#     val1_norm = math.sqrt(val1_norm)
#     val2_norm = math.sqrt(val2_norm)

#     distance = distance / (val1_norm * val2_norm)
    
    return distance

calculate_distance(tf_idf, news_ids[2], news_ids[1])

42.495038054961356

In [22]:
def get_k_most_similar_news(tf_idf, n_id, k):
    distances = {}
    for news_id in tf_idf.keys():
        if news_id != n_id:
            distances[news_id] = calculate_distance(tf_idf, news_id, n_id)
    return dict(sorted(distances.items(), key=lambda x:x[1], reverse=True)[:k]).keys()

def print_k_most_similar_news(tf_idf, n_id, k, corpus, news_indices):
    similar = get_k_most_similar_news(tf_idf, n_id, k)
    print(f'id: {n_id}, text: {corpus[news_indices[n_id]]}')
    print(f'\n{k} most similar:')
    for s_id in similar:
        print(f'\nid: {s_id}, text: {corpus[news_indices[s_id]]}')

print_k_most_similar_news(tf_idf, news_ids[42337], 5, texts, news_indices)

id: N58544, text: A MAN claims he has created a car that might solve the world's traffic congestion problems. Rick Woodbury from Spokane, Washington USA, is the president, founder and sole employee of 'Commuter Cars.' The carmaker's flagship model is the 2005 super slim two-seater Tango T600, a high-performance electric car that preceded Tesla. Rick told BTV: "I started this company 21 years ago   it was based on an idea that I came up with in 1982." He was inspired by the shocking traffic congestion he had to face on a daily basis. "I used to drive a Porsche from Beverly Hills to Hermosa Beach every day and the traffic was horrendous," explained Rick. What really made Rick think about a solution was the fact that in most of the cars he would see in his commute were occupied by lone drivers. "I noticed that everybody around me was a single occupant in a car, taking up the whole lane," Rick said. Living and working in Los Angeles also helped inspire Rick's unique creation. "I thought, e